## How does one determine the efficiency of an indicator/signal?

Currently, it seems hard that I could use the patterns alone in determining the trades (without understanding the patterns).
So, what I can do is:

1. Let the code run on daily basis and generate signals in the time frame (if daily, then for a month's data etc). 
   Print a summary of signals generated on the day, and manually evaluate them. (Lesser time to code, so we'll do that.)
2. Try to also estimate the accuracy/reliability of the candlestick myself rather than relying on someone else's 
   blackbox evaluation. The question is, how to deduce the efficacy (what time frame)? One stab at the answer would be, 
   understand each indicator and have its custom window determined empirically or via text.

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
#Import basic libraries
%matplotlib inline

import os
import sys
import settings
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from django_pandas.io import read_frame
from mplfinance.original_flavor import candlestick_ohlc
import matplotlib.dates as mpl_dates

from matplotlib.dates import date2num

# imports
import pandas_datareader.data as pdr
import datetime
import talib
from talib.abstract import *
from talib import MA_Type

# format price data
pd.options.display.float_format = '{:0.2f}'.format

In [ ]:
#Prepare to load stock data as pandas dataframe from source. In this case, prepare django
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

from stocks.models import Listing, Stock

In [ ]:
def evaluate_signal(df, signal_val, signal_frequency, success_frequency, tolerance=2):
    '''
    For the signal value passed, determine if price moves in the direction of signal 
    or not within the tolerance period and update the probability of success metric (frequentist)
    '''
    occurances = df[df[signal_val]!=0]
    #print(occurances.tail())
    idxs = []
    for idx, occurance in occurances.iterrows():
        #print(df.index.get_loc(idx))
        idxs.append(df.index.get_loc(idx))

    for idx in idxs:
        #print(idx)
        if df.iloc[idx][signal_val] > 0:
            #Bullish
            signal_frequency['bullish'][signal_val] += 1
            if idx+tolerance <= len(df)-1 and df.iloc[idx+tolerance]['close']> df.iloc[idx]['close']:
                #Pattern is success
                success_frequency['bullish'][signal_val] +=1
        elif df.iloc[idx][signal_val] < 0:
            #Bearish
            signal_frequency['bearish'][signal_val] += 1
            if idx+tolerance <= len(df)-1 and df.iloc[idx+tolerance]['close']< df.iloc[idx]['close']:
                #Pattern is success
                success_frequency['bearish'][signal_val] +=1
    

In [ ]:
#For all stocks, for all days, run the above function once to determine the score of each indicator
# create columns for each pattern
candle_names = talib.get_function_groups()['Pattern Recognition']

signal_frequency = {'bullish': {}, 'bearish':{}}
success_frequency = {'bullish': {}, 'bearish':{}}

for candle in candle_names:
    signal_frequency['bullish'][candle] = 0
    success_frequency['bullish'][candle] = 0
    signal_frequency['bearish'][candle] = 0
    success_frequency['bearish'][candle] = 0
    
#Asian paints
#stock = Stock.objects.get(sid='ASIANPAINT')
#listing = get_stock_listing(stock, duration=-1, last_date = datetime.date.today())
#for candle in candle_names:
#    listing[candle] = getattr(talib, candle)(listing['open'], listing['high'], listing['low'], listing['close'])
#    evaluate_signal(listing, candle, signal_frequency, success_frequency)
    
for stock in Stock.objects.all():
    #listing = get_stock_listing(stock, duration=-1, last_date = datetime.date.today(), resample=False)
    listing = get_stock_listing(stock, duration=-1, last_date = datetime.date.today(), resample=False, monthly=False) #For weekly/monthly charts
    if len(listing)==0:
        continue
    for candle in candle_names:
        listing[candle] = getattr(talib, candle)(listing['open'], listing['high'], listing['low'], listing['close'])
        evaluate_signal(listing, candle, signal_frequency, success_frequency, tolerance=2)

In [ ]:
print('Bullish')
for key in success_frequency['bullish']:
    if signal_frequency['bullish'][key] !=0:
        print(f"{key}: {success_frequency['bullish'][key]/signal_frequency['bullish'][key]}")
    else:
        print(f'{key}: NA')
print('Bearish')
for key in success_frequency['bearish']:
    if signal_frequency['bearish'][key] !=0:
        print(f"{key}: {success_frequency['bearish'][key]/signal_frequency['bearish'][key]}")
    else:
        print(f'{key}: NA')

A naive evaluation of candlesticks shows the reliability of most patterns is 50-50:

|Candle | Probability of success (tolerance T+2) |
|---|---|
| CDL2CROWS |  0.5460467870891323 |
| CDL3BLACKCROWS |  0.5070257611241218 |
| CDL3INSIDE |  0.49692876836631417 |
| CDL3LINESTRIKE |  0.5453015427769986 |
| CDL3OUTSIDE |  0.5025643409411363 |
| CDL3STARSINSOUTH |  0.494949494949495 |
| CDL3WHITESOLDIERS |  0.5526881720430108 |
| CDLABANDONEDBABY |  0.45072463768115945 |
| CDLADVANCEBLOCK |  0.5127080521817364 |
| CDLBELTHOLD |  0.48373096544933447 |
| CDLBREAKAWAY |  0.46296296296296297 |
| CDLCLOSINGMARUBOZU |  0.47412948411472927 |
| CDLCONCEALBABYSWALL |  0.375 |
| CDLCOUNTERATTACK |  0.4716149401322452 |
| CDLDARKCLOUDCOVER |  0.5075513006048462 |
| CDLDOJI |  0.4588538885253464 |
| CDLDOJISTAR |  0.42915841186233117 |
| CDLDRAGONFLYDOJI |  0.48787842740736337 |
| CDLENGULFING |  0.48335156519263334 |
| CDLEVENINGDOJISTAR |  0.5011031439602868 |
| CDLEVENINGSTAR |  0.5136806391594615 |
| CDLGAPSIDESIDEWHITE |  0.47496998685138253 |
| CDLGRAVESTONEDOJI |  0.4761196673877917 |
| CDLHAMMER |  0.4344750634673655 |
| CDLHANGINGMAN |  0.4822786186714649 |
| CDLHARAMI |  0.48793646529944074 |
| CDLHARAMICROSS |  0.4953989364851901 |
| CDLHIGHWAVE |  0.48989004299950833 |
| CDLHIKKAKE |  0.47799536454458336 |
| CDLHIKKAKEMOD |  0.46798071957307624 |
| CDLHOMINGPIGEON |  0.45778711916000037 |
| CDLIDENTICAL3CROWS |  0.5525894287239722 |
| CDLINNECK |  0.5170970705317954 |
| CDLINVERTEDHAMMER |  0.4820038195974732 |
| CDLKICKING |  0.5548281505728314 |
| CDLKICKINGBYLENGTH |  0.5073649754500819 |
| CDLLADDERBOTTOM |  0.472875226039783 |
| CDLLONGLEGGEDDOJI |  0.47349946064202547 |
| CDLLONGLINE |  0.4816818213781502 |
| CDLMARUBOZU |  0.47189488499886223 |
| CDLMATCHINGLOW |  0.4688532860737224 |
| CDLMATHOLD |  0.5833333333333334 |
| CDLMORNINGDOJISTAR |  0.42487125113601937 |
| CDLMORNINGSTAR |  0.4322971386193684 |
| CDLONNECK |  0.5044750846637639 |
| CDLPIERCING |  0.4209733825529435 |
| CDLRICKSHAWMAN |  0.4647953549482644 |
| CDLRISEFALL3METHODS |  0.4723809523809524 |
| CDLSEPARATINGLINES |  0.5114880535030473 |
| CDLSHOOTINGSTAR |  0.5013450418972821 |
| CDLSHORTLINE |  0.4797533005546477 |
| CDLSPINNINGTOP |  0.49113158718038774 |
| CDLSTALLEDPATTERN |  0.3833630686886708 |
| CDLSTICKSANDWICH |  0.4902059773828756 |
| CDLTAKURI |  0.4908589818206717 |
| CDLTASUKIGAP |  0.4940449131739249 |
| CDLTHRUSTING |  0.5516593781021114 |
| CDLTRISTAR |  0.503246624436304 |
| CDLUNIQUE3RIVER |  0.44768454837230626 |
| CDLUPSIDEGAP2CROWS |  0.5627996164908916 |
| CDLXSIDEGAP3METHODS |  0.5069464544138929 |


|Bullish Candle | Probability of success (tolerance T+1) |
|---|---|
|CDL2CROWS| NA|
|CDL3BLACKCROWS| NA|
|CDL3INSIDE| 0.5093949002856469|
|CDL3LINESTRIKE| 0.5080645161290323|
|CDL3OUTSIDE| 0.47166355887053807|
|CDL3STARSINSOUTH| 0.42424242424242425|
|CDL3WHITESOLDIERS| 0.585923544122901|
|CDLABANDONEDBABY| 0.4434782608695652|
|CDLADVANCEBLOCK| NA|
|CDLBELTHOLD| 0.46064423176523395|
|CDLBREAKAWAY| 0.4410480349344978|
|CDLCLOSINGMARUBOZU| 0.46498389378419375|
|CDLCONCEALBABYSWALL| 0.38333333333333336|
|CDLCOUNTERATTACK| 0.414287611207011|
|CDLDARKCLOUDCOVER| NA|
|CDLDOJI| 0.46285214806169467|
|CDLDOJISTAR| 0.4148882493646285|
|CDLDRAGONFLYDOJI| 0.4993532155275744|
|CDLENGULFING| 0.4457307389762085|
|CDLEVENINGDOJISTAR| NA|
|CDLEVENINGSTAR| NA|
|CDLGAPSIDESIDEWHITE| 0.3941828368969637|
|CDLGRAVESTONEDOJI| 0.4751300895874685|
|CDLHAMMER| 0.42932697717717105|
|CDLHANGINGMAN| NA|
|CDLHARAMI| 0.465604013463287|
|CDLHARAMICROSS| 0.47128139584422624|
|CDLHIGHWAVE| 0.46165536211271824|
|CDLHIKKAKE| 0.4552668673755286|
|CDLHIKKAKEMOD| 0.45287698412698413|
|CDLHOMINGPIGEON| 0.4555541477365287|
|CDLIDENTICAL3CROWS| NA|
|CDLINNECK| NA|
|CDLINVERTEDHAMMER| 0.4836253675912923|
|CDLKICKING| 0.5249266862170088|
|CDLKICKINGBYLENGTH| 0.5076923076923077|
|CDLLADDERBOTTOM| 0.480225988700565|
|CDLLONGLEGGEDDOJI| 0.47445564836562065|
|CDLLONGLINE| 0.45853702611706076|
|CDLMARUBOZU| 0.45616529421032204|
|CDLMATCHINGLOW| 0.4650073338890289|
|CDLMATHOLD| 0.5694444444444444|
|CDLMORNINGDOJISTAR| 0.4336966394187103|
|CDLMORNINGSTAR| 0.44618044834187615|
|CDLONNECK| NA|
|CDLPIERCING| 0.43491339575954585|
|CDLRICKSHAWMAN| 0.46151988407663824|
|CDLRISEFALL3METHODS| 0.4742268041237113|
|CDLSEPARATINGLINES| 0.4854368932038835|
|CDLSHOOTINGSTAR| NA|
|CDLSHORTLINE| 0.4609552895788189|
|CDLSPINNINGTOP| 0.46204914008410597|
|CDLSTALLEDPATTERN| NA|
|CDLSTICKSANDWICH| 0.49142453591606133|
|CDLTAKURI| 0.5023481661533907|
|CDLTASUKIGAP| 0.4621153846153846|
|CDLTHRUSTING| NA|
|CDLTRISTAR| 0.4771017209508188|
|CDLUNIQUE3RIVER| 0.45475382781699825|
|CDLUPSIDEGAP2CROWS| NA|
|CDLXSIDEGAP3METHODS| 0.4595813785636954|


|Bearish Candle | Probability of success (tolerance T+1) |
|---|---|
|CDL2CROWS| 0.5353818880189097|
|CDL3BLACKCROWS| 0.5134660421545667|
|CDL3INSIDE| 0.48872689403801095|
|CDL3LINESTRIKE| 0.5533104041272571|
|CDL3OUTSIDE| 0.5139048872799078|
|CDL3STARSINSOUTH| NA|
|CDL3WHITESOLDIERS| NA|
|CDLABANDONEDBABY| 0.45507246376811594|
|CDLADVANCEBLOCK| 0.4975374972017014|
|CDLBELTHOLD| 0.49741535934661185|
|CDLBREAKAWAY| 0.5123152709359606|
|CDLCLOSINGMARUBOZU| 0.4781608667707281|
|CDLCONCEALBABYSWALL| NA|
|CDLCOUNTERATTACK| 0.5063522825150732|
|CDLDARKCLOUDCOVER| 0.49382716049382713|
|CDLDOJI| NA|
|CDLDOJISTAR| 0.41627922518067345|
|CDLDRAGONFLYDOJI| NA|
|CDLENGULFING| 0.502459795774939|
|CDLEVENINGDOJISTAR| 0.49442071910731505|
|CDLEVENINGSTAR| 0.508554249795026|
|CDLGAPSIDESIDEWHITE| 0.4572430154739563|
|CDLGRAVESTONEDOJI| NA|
|CDLHAMMER| NA|
|CDLHANGINGMAN| 0.4537945174422616|
|CDLHARAMI| 0.5121602884082085|
|CDLHARAMICROSS| 0.5135089907810265|
|CDLHIGHWAVE| 0.5200587560481055|
|CDLHIKKAKE| 0.4947362289198347|
|CDLHIKKAKEMOD| 0.4901685393258427|
|CDLHOMINGPIGEON| NA|
|CDLIDENTICAL3CROWS| 0.5373532550693704|
|CDLINNECK| 0.5023464599061416|
|CDLINVERTEDHAMMER| NA|
|CDLKICKING| 0.4666666666666667|
|CDLKICKINGBYLENGTH| 0.458041958041958|
|CDLLADDERBOTTOM| NA|
|CDLLONGLEGGEDDOJI| NA|
|CDLLONGLINE| 0.4949045795716338|
|CDLMARUBOZU| 0.47500719562505994|
|CDLMATCHINGLOW| NA|
|CDLMATHOLD| NA|
|CDLMORNINGDOJISTAR| NA|
|CDLMORNINGSTAR| NA|
|CDLONNECK| 0.4898809883404821|
|CDLPIERCING| NA|
|CDLRICKSHAWMAN| NA|
|CDLRISEFALL3METHODS| 0.5072463768115942|
|CDLSEPARATINGLINES| 0.49647204776304565|
|CDLSHOOTINGSTAR| 0.4993051909952753|
|CDLSHORTLINE| 0.5087696905080114|
|CDLSPINNINGTOP| 0.5185577225162522|
|CDLSTALLEDPATTERN| 0.34155281572809065|
|CDLSTICKSANDWICH| NA|
|CDLTAKURI| NA|
|CDLTASUKIGAP| 0.5255110805703488|
|CDLTHRUSTING| 0.5362330492576513|
|CDLTRISTAR| 0.5012488921118363|
|CDLUNIQUE3RIVER| NA|
|CDLUPSIDEGAP2CROWS| 0.5544152744630072|
|CDLXSIDEGAP3METHODS| 0.5283109984305203|


|Bullish Candle | Probability of success (tolerance T+2) |
|---|---|
|CDL2CROWS| NA|
|CDL3BLACKCROWS| NA|
|CDL3INSIDE| 0.4990696821195524|
|CDL3LINESTRIKE| 0.5209677419354839|
|CDL3OUTSIDE| 0.4610415556739478|
|CDL3STARSINSOUTH| 0.494949494949495|
|CDL3WHITESOLDIERS| 0.5512683101107538|
|CDLABANDONEDBABY| 0.4318840579710145|
|CDLADVANCEBLOCK| NA|
|CDLBELTHOLD| 0.45384062451935303|
|CDLBREAKAWAY| 0.43231441048034935|
|CDLCLOSINGMARUBOZU| 0.4500849340128054|
|CDLCONCEALBABYSWALL| 0.375|
|CDLCOUNTERATTACK| 0.41880228389324126|
|CDLDARKCLOUDCOVER| NA|
|CDLDOJI| 0.4590514879865956|
|CDLDOJISTAR| 0.4130942592315742|
|CDLDRAGONFLYDOJI| 0.4883163428788436|
|CDLENGULFING| 0.4389257461745106|
|CDLEVENINGDOJISTAR| NA|
|CDLEVENINGSTAR| NA|
|CDLGAPSIDESIDEWHITE| 0.42009640341119764|
|CDLGRAVESTONEDOJI| 0.4762727321495628|
|CDLHAMMER| 0.43438053052080244|
|CDLHANGINGMAN| NA|
|CDLHARAMI| 0.4662819633428512|
|CDLHARAMICROSS| 0.470526152714034|
|CDLHIGHWAVE| 0.4624747768268075|
|CDLHIKKAKE| 0.4566416650714373|
|CDLHIKKAKEMOD| 0.45349702380952384|
|CDLHOMINGPIGEON| 0.45800676959834924|
|CDLIDENTICAL3CROWS| NA|
|CDLINNECK| NA|
|CDLINVERTEDHAMMER| 0.4819519403437865|
|CDLKICKING| 0.5806451612903226|
|CDLKICKINGBYLENGTH| 0.5353846153846153|
|CDLLADDERBOTTOM| 0.47480225988700564|
|CDLLONGLEGGEDDOJI| 0.4737543170975986|
|CDLLONGLINE| 0.45043845466131516|
|CDLMARUBOZU| 0.4438881182297811|
|CDLMATCHINGLOW| 0.46881320229256723|
|CDLMATHOLD| 0.5833333333333334|
|CDLMORNINGDOJISTAR| 0.4265818952467454|
|CDLMORNINGSTAR| 0.4337059223121102|
|CDLONNECK| NA|
|CDLPIERCING| 0.4220561835912862|
|CDLRICKSHAWMAN| 0.46498685128535394|
|CDLRISEFALL3METHODS| 0.4536082474226804|
|CDLSEPARATINGLINES| 0.47555163283318624|
|CDLSHOOTINGSTAR| NA|
|CDLSHORTLINE| 0.4580559269932532|
|CDLSPINNINGTOP| 0.4633773786737063|
|CDLSTALLEDPATTERN| NA|
|CDLSTICKSANDWICH| 0.4898103309120258|
|CDLTAKURI| 0.49127051443339464|
|CDLTASUKIGAP| 0.4542307692307692|
|CDLTHRUSTING| NA|
|CDLTRISTAR| 0.4611679554271134|
|CDLUNIQUE3RIVER| 0.4485192995324104|
|CDLUPSIDEGAP2CROWS| NA|
|CDLXSIDEGAP3METHODS| 0.46192710212919524|


|Bearish Candle | Probability of success (tolerance T+2) |
|---|---|
|CDL2CROWS| 0.5451322204166051|
|CDL3BLACKCROWS| 0.5070257611241218|
|CDL3INSIDE| 0.49643322051549077|
|CDL3LINESTRIKE| 0.55932932072227|
|CDL3OUTSIDE| 0.5222448277409607|
|CDL3STARSINSOUTH| NA|
|CDL3WHITESOLDIERS| NA|
|CDLABANDONEDBABY| 0.46956521739130436|
|CDLADVANCEBLOCK| 0.5109693306469666|
|CDLBELTHOLD| 0.5056862094374539|
|CDLBREAKAWAY| 0.4975369458128079|
|CDLCLOSINGMARUBOZU| 0.49422918005692773|
|CDLCONCEALBABYSWALL| NA|
|CDLCOUNTERATTACK| 0.5133505598621878|
|CDLDARKCLOUDCOVER| 0.507396285174063|
|CDLDOJI| NA|
|CDLDOJISTAR| 0.445301536082179|
|CDLDRAGONFLYDOJI| NA|
|CDLENGULFING| 0.5061579455725556|
|CDLEVENINGDOJISTAR| 0.5006199200991872|
|CDLEVENINGSTAR| 0.5130910084722602|
|CDLGAPSIDESIDEWHITE| 0.5222902871711341|
|CDLGRAVESTONEDOJI| NA|
|CDLHAMMER| NA|
|CDLHANGINGMAN| 0.4815473784857416|
|CDLHARAMI| 0.5261716583471991|
|CDLHARAMICROSS| 0.5349286518392309|
|CDLHIGHWAVE| 0.5218620756801605|
|CDLHIKKAKE| 0.5020760584985712|
|CDLHIKKAKEMOD| 0.5025280898876404|
|CDLHOMINGPIGEON| NA|
|CDLIDENTICAL3CROWS| 0.5533617929562433|
|CDLINNECK| 0.5171393593144257|
|CDLINVERTEDHAMMER| NA|
|CDLKICKING| 0.5222222222222223|
|CDLKICKINGBYLENGTH| 0.4755244755244755|
|CDLLADDERBOTTOM| NA|
|CDLLONGLEGGEDDOJI| NA|
|CDLLONGLINE| 0.5013853106328934|
|CDLMARUBOZU| 0.49439041063033673|
|CDLMATCHINGLOW| NA|
|CDLMATHOLD| NA|
|CDLMORNINGDOJISTAR| NA|
|CDLMORNINGSTAR| NA|
|CDLONNECK| 0.5040778106687609|
|CDLPIERCING| NA|
|CDLRICKSHAWMAN| NA|
|CDLRISEFALL3METHODS| 0.5217391304347826|
|CDLSEPARATINGLINES| 0.5279134682484299|
|CDLSHOOTINGSTAR| 0.5010653738072446|
|CDLSHORTLINE| 0.5147519258780953|
|CDLSPINNINGTOP| 0.5192343942579569|
|CDLSTALLEDPATTERN| 0.38231700544263025|
|CDLSTICKSANDWICH| NA|
|CDLTAKURI| NA|
|CDLTASUKIGAP| 0.5279161656072839|
|CDLTHRUSTING| 0.551294420324811|
|CDLTRISTAR| 0.5468804555098972|
|CDLUNIQUE3RIVER| NA|
|CDLUPSIDEGAP2CROWS| 0.5608591885441527|
|CDLXSIDEGAP3METHODS| 0.537124230351322|

Evaluation (naive) on weekly basis might yield better results for a few patterns. Let's check:

|Candle | Probability of success (tolerance T+1) |
|---|---|
|CDL2CROWS| 0.5224056603773585|
|CDL3BLACKCROWS| 0.4863636363636364|
|CDL3INSIDE| 0.4783989114412065|
|CDL3LINESTRIKE| 0.5295566502463054|
|CDL3OUTSIDE| 0.4906260717143619|
|CDL3STARSINSOUTH| 0.5|
|CDL3WHITESOLDIERS| 0.531578947368421|
|CDLABANDONEDBABY| 0.6491228070175439|
|CDLADVANCEBLOCK| 0.5042065564258775|
|CDLBELTHOLD| 0.4943809832123202|
|CDLBREAKAWAY| 0.43617021276595747|
|CDLCLOSINGMARUBOZU| 0.5134032052348472|
|CDLCONCEALBABYSWALL| 0.16666666666666666|
|CDLCOUNTERATTACK| 0.5133876600698487|
|CDLDARKCLOUDCOVER| 0.5394029850746269|
|CDLDOJI| 0.47130831074840646|
|CDLDOJISTAR| 0.5052018759096545|
|CDLDRAGONFLYDOJI| 0.47158629647670075|
|CDLENGULFING| 0.4906242404980463|
|CDLEVENINGDOJISTAR| 0.5424224692802808|
|CDLEVENINGSTAR| 0.5296682005328167|
|CDLGAPSIDESIDEWHITE| 0.4715447154471545|
|CDLGRAVESTONEDOJI| 0.4862716049382716|
|CDLHAMMER| 0.48901654167331526|
|CDLHANGINGMAN| 0.505983268029228|
|CDLHARAMI| 0.4811628539247291|
|CDLHARAMICROSS| 0.47673499499394|
|CDLHIGHWAVE| 0.4908505086942056|
|CDLHIKKAKE| 0.480273131616864|
|CDLHIKKAKEMOD| 0.4737827715355805|
|CDLHOMINGPIGEON| 0.46464832933357947|
|CDLIDENTICAL3CROWS| 0.559254327563249|
|CDLINNECK| 0.5066258919469928|
|CDLINVERTEDHAMMER| 0.4956948098541019|
|CDLKICKING| 0.3333333333333333|
|CDLKICKINGBYLENGTH| 0.5|
|CDLLADDERBOTTOM| 0.4651773981603154|
|CDLLONGLEGGEDDOJI| 0.4754477541962304|
|CDLLONGLINE| 0.5007549349086077|
|CDLMARUBOZU| 0.5164319248826291|
|CDLMATCHINGLOW| 0.47549410264583997|
|CDLMATHOLD| 0.3125|
|CDLMORNINGDOJISTAR| 0.43947175348495965|
|CDLMORNINGSTAR| 0.4513064133016627|
|CDLONNECK| 0.48604651162790696|
|CDLPIERCING| 0.45704845814977973|
|CDLRICKSHAWMAN| 0.4798185267295957|
|CDLRISEFALL3METHODS| 0.3967611336032389|
|CDLSEPARATINGLINES| 0.48154136351387766|
|CDLSHOOTINGSTAR| 0.5273487773487774|
|CDLSHORTLINE| 0.49716025229223854|
|CDLSPINNINGTOP| 0.49152026648303254|
|CDLSTALLEDPATTERN| 0.5280437756497948|
|CDLSTICKSANDWICH| 0.5035750766087844|
|CDLTAKURI| 0.4719309566059042|
|CDLTASUKIGAP| 0.48905690409868685|
|CDLTHRUSTING| 0.5282399812514648|
|CDLTRISTAR| 0.48233782915863843|
|CDLUNIQUE3RIVER| 0.45666666666666667|
|CDLUPSIDEGAP2CROWS| 0.5017543859649123|
|CDLXSIDEGAP3METHODS| 0.46954625970840713|

Weekly patterns with bullish and bearish biases separate

|Bullish Candle | Probability of success (tolerance T+1) |
|---|---|
|CDL2CROWS| NA|
|CDL3BLACKCROWS| NA|
|CDL3INSIDE| 0.46298554258839925|
|CDL3LINESTRIKE| 0.5098039215686274|
|CDL3OUTSIDE| 0.46309023485784917|
|CDL3STARSINSOUTH| 0.5|
|CDL3WHITESOLDIERS| 0.531578947368421|
|CDLABANDONEDBABY| 0.6470588235294118|
|CDLADVANCEBLOCK| NA|
|CDLBELTHOLD| 0.4717330164984735|
|CDLBREAKAWAY| 0.4489795918367347|
|CDLCLOSINGMARUBOZU| 0.5107821908462611|
|CDLCONCEALBABYSWALL| 0.16666666666666666|
|CDLCOUNTERATTACK| 0.4794952681388013|
|CDLDARKCLOUDCOVER| NA|
|CDLDOJI| 0.47130831074840646|
|CDLDOJISTAR| 0.4844294425087108|
|CDLDRAGONFLYDOJI| 0.47158629647670075|
|CDLENGULFING| 0.46978073027966183|
|CDLEVENINGDOJISTAR| NA|
|CDLEVENINGSTAR| NA|
|CDLGAPSIDESIDEWHITE| 0.4806800618238022|
|CDLGRAVESTONEDOJI| 0.4862716049382716|
|CDLHAMMER| 0.48901654167331526|
|CDLHANGINGMAN| NA|
|CDLHARAMI| 0.46408686825186873|
|CDLHARAMICROSS| 0.4620573355817875|
|CDLHIGHWAVE| 0.4679957496331529|
|CDLHIKKAKE| 0.46556356634275226|
|CDLHIKKAKEMOD| 0.46079295154185024|
|CDLHOMINGPIGEON| 0.46464832933357947|
|CDLIDENTICAL3CROWS| NA|
|CDLINNECK| NA|
|CDLINVERTEDHAMMER| 0.4956948098541019|
|CDLKICKING| 0.5|
|CDLKICKINGBYLENGTH| 0.6666666666666666|
|CDLLADDERBOTTOM| 0.4651773981603154|
|CDLLONGLEGGEDDOJI| 0.4754477541962304|
|CDLLONGLINE| 0.49211397296747594|
|CDLMARUBOZU| 0.5265671240247511|
|CDLMATCHINGLOW| 0.47549410264583997|
|CDLMATHOLD| 0.3125|
|CDLMORNINGDOJISTAR| 0.43947175348495965|
|CDLMORNINGSTAR| 0.4513064133016627|
|CDLONNECK| NA|
|CDLPIERCING| 0.45704845814977973|
|CDLRICKSHAWMAN| 0.4798185267295957|
|CDLRISEFALL3METHODS| 0.3655913978494624|
|CDLSEPARATINGLINES| 0.4557700942972609|
|CDLSHOOTINGSTAR| NA|
|CDLSHORTLINE| 0.48835943876559523|
|CDLSPINNINGTOP| 0.46840034362438887|
|CDLSTALLEDPATTERN| NA|
|CDLSTICKSANDWICH| 0.5035750766087844|
|CDLTAKURI| 0.4719309566059042|
|CDLTASUKIGAP| 0.4704904405652535|
|CDLTHRUSTING| NA|
|CDLTRISTAR| 0.45887961859356374|
|CDLUNIQUE3RIVER| 0.45666666666666667|
|CDLUPSIDEGAP2CROWS| NA|
|CDLXSIDEGAP3METHODS| 0.4237488895469351|

|Bearish Candle | Probability of success (tolerance T+1) |
|---|---|
|CDL2CROWS| 0.5224056603773585|
|CDL3BLACKCROWS| 0.4863636363636364|
|CDL3INSIDE| 0.5071474983755686|
|CDL3LINESTRIKE| 0.5431841831425598|
|CDL3OUTSIDE| 0.507888782120827|
|CDL3STARSINSOUTH| NA|
|CDL3WHITESOLDIERS| NA|
|CDLABANDONEDBABY| 0.65|
|CDLADVANCEBLOCK| 0.5042065564258775|
|CDLBELTHOLD| 0.512124425592082|
|CDLBREAKAWAY| 0.4222222222222222|
|CDLCLOSINGMARUBOZU| 0.5149684687876426|
|CDLCONCEALBABYSWALL| NA|
|CDLCOUNTERATTACK| 0.533210332103321|
|CDLDARKCLOUDCOVER| 0.5394029850746269|
|CDLDOJI| NA|
|CDLDOJISTAR| 0.5255684851072916|
|CDLDRAGONFLYDOJI| NA|
|CDLENGULFING| 0.5046616400212746|
|CDLEVENINGDOJISTAR| 0.5424224692802808|
|CDLEVENINGSTAR| 0.5296682005328167|
|CDLGAPSIDESIDEWHITE| 0.45671267252195735|
|CDLGRAVESTONEDOJI| NA|
|CDLHAMMER| NA|
|CDLHANGINGMAN| 0.505983268029228|
|CDLHARAMI| 0.5104029496971293|
|CDLHARAMICROSS| 0.5047516860821581|
|CDLHIGHWAVE| 0.5155211652253072|
|CDLHIKKAKE| 0.4989616817910278|
|CDLHIKKAKEMOD| 0.5053533190578159|
|CDLHOMINGPIGEON| NA|
|CDLIDENTICAL3CROWS| 0.559254327563249|
|CDLINNECK| 0.5066258919469928|
|CDLINVERTEDHAMMER| NA|
|CDLKICKING| 0.25|
|CDLKICKINGBYLENGTH| 0.3333333333333333|
|CDLLADDERBOTTOM| NA|
|CDLLONGLEGGEDDOJI| NA|
|CDLLONGLINE| 0.505968450880207|
|CDLMARUBOZU| 0.5109981249098514|
|CDLMATCHINGLOW| NA|
|CDLMATHOLD| NA|
|CDLMORNINGDOJISTAR| NA|
|CDLMORNINGSTAR| NA|
|CDLONNECK| 0.48604651162790696|
|CDLPIERCING| NA|
|CDLRICKSHAWMAN| NA|
|CDLRISEFALL3METHODS| 0.4918032786885246|
|CDLSEPARATINGLINES| 0.5202156334231806|
|CDLSHOOTINGSTAR| 0.5273487773487774|
|CDLSHORTLINE| 0.5024553936814536|
|CDLSPINNINGTOP| 0.5149505588993981|
|CDLSTALLEDPATTERN| 0.5280437756497948|
|CDLSTICKSANDWICH| NA|
|CDLTAKURI| NA|
|CDLTASUKIGAP| 0.5061068702290077|
|CDLTHRUSTING| 0.5282399812514648|
|CDLTRISTAR| 0.5097493036211699|
|CDLUNIQUE3RIVER| NA|
|CDLUPSIDEGAP2CROWS| 0.5017543859649123|
|CDLXSIDEGAP3METHODS| 0.5085815244825845|

Finally on Monthly charts:

|Candle | Probability of success (tolerance T+1) |
|---|---|
|CDL2CROWS| 0.47619047619047616|
|CDL3BLACKCROWS| 0.43902439024390244|
|CDL3INSIDE| 0.4763636363636364|
|CDL3LINESTRIKE| 0.47162162162162163|
|CDL3OUTSIDE| 0.44014494629222206|
|CDL3STARSINSOUTH| 0.0|
|CDL3WHITESOLDIERS| 0.5238095238095238|
|CDLABANDONEDBABY| 0.5294117647058824|
|CDLADVANCEBLOCK| 0.4509545753785385|
|CDLBELTHOLD| 0.4747225647348952|
|CDLBREAKAWAY| 0.75|
|CDLCLOSINGMARUBOZU| 0.4944698916322199|
|CDLCONCEALBABYSWALL| 0.3333333333333333|
|CDLCOUNTERATTACK| 0.4418604651162791|
|CDLDARKCLOUDCOVER| 0.5505882352941176|
|CDLDOJI| 0.4935917779459512|
|CDLDOJISTAR| 0.5326938136256852|
|CDLDRAGONFLYDOJI| 0.48925619834710743|
|CDLENGULFING| 0.5061408967346805|
|CDLEVENINGDOJISTAR| 0.5009596928982726|
|CDLEVENINGSTAR| 0.5244807121661721|
|CDLGAPSIDESIDEWHITE| 0.45539033457249073|
|CDLGRAVESTONEDOJI| 0.49532418952618457|
|CDLHAMMER| 0.5021872265966754|
|CDLHANGINGMAN| 0.49727520435967304|
|CDLHARAMI| 0.4861380648738564|
|CDLHARAMICROSS| 0.4975456508933831|
|CDLHIGHWAVE| 0.4895836464752743|
|CDLHIKKAKE| 0.48177253383170837|
|CDLHIKKAKEMOD| 0.4945736434108527|
|CDLHOMINGPIGEON| 0.5374554102259215|
|CDLIDENTICAL3CROWS| 0.5727170236753101|
|CDLINNECK| 0.48823529411764705|
|CDLINVERTEDHAMMER| 0.5385985748218527|
|CDLKICKING| NA|
|CDLKICKINGBYLENGTH| NA|
|CDLLADDERBOTTOM| 0.484375|
|CDLLONGLEGGEDDOJI| 0.503790148219542|
|CDLLONGLINE| 0.4792579425681596|
|CDLMARUBOZU| 0.48343373493975905|
|CDLMATCHINGLOW| 0.4993608862377503|
|CDLMATHOLD| 0.6666666666666666|
|CDLMORNINGDOJISTAR| 0.42769857433808556|
|CDLMORNINGSTAR| 0.4146981627296588|
|CDLONNECK| 0.421875|
|CDLPIERCING| 0.4808743169398907|
|CDLRICKSHAWMAN| 0.5107363131043935|
|CDLRISEFALL3METHODS| 0.5555555555555556|
|CDLSEPARATINGLINES| 0.4719014626635874|
|CDLSHOOTINGSTAR| 0.4926727145847872|
|CDLSHORTLINE| 0.4651881720430108|
|CDLSPINNINGTOP| 0.4889786455597616|
|CDLSTALLEDPATTERN| 0.4568245125348189|
|CDLSTICKSANDWICH| 0.4005602240896359|
|CDLTAKURI| 0.4860824742268041|
|CDLTASUKIGAP| 0.5050632911392405|
|CDLTHRUSTING| 0.4918918918918919|
|CDLTRISTAR| 0.4742489270386266|
|CDLUNIQUE3RIVER| 0.45977011494252873|
|CDLUPSIDEGAP2CROWS| 0.4482758620689655|
|CDLXSIDEGAP3METHODS| 0.46965784377017433|


Monthly patterns considering bullish and bearish signals

Another naive implementation that considers the bearish and bullish signals separately (if some pattern is more reliable in a particular direction)

|Bullish Candle | Probability of success (tolerance T+1) |
|---|---|
|CDL2CROWS| NA|
|CDL3BLACKCROWS| NA|
|CDL3INSIDE| 0.4897959183673469|
|CDL3LINESTRIKE| 0.4481132075471698|
|CDL3OUTSIDE| 0.46178861788617886|
|CDL3STARSINSOUTH| 0.0|
|CDL3WHITESOLDIERS| 0.5238095238095238|
|CDLABANDONEDBABY| 0.6|
|CDLADVANCEBLOCK| NA|
|CDLBELTHOLD| 0.4685317835743569|
|CDLBREAKAWAY| 0.3333333333333333|
|CDLCLOSINGMARUBOZU| 0.4913907284768212|
|CDLCONCEALBABYSWALL| 0.3333333333333333|
|CDLCOUNTERATTACK| 0.5294117647058824|
|CDLDARKCLOUDCOVER| NA|
|CDLDOJI| 0.4935917779459512|
|CDLDOJISTAR| 0.5801282051282052|
|CDLDRAGONFLYDOJI| 0.48925619834710743|
|CDLENGULFING| 0.48058138626240526|
|CDLEVENINGDOJISTAR| NA|
|CDLEVENINGSTAR| NA|
|CDLGAPSIDESIDEWHITE| 0.46437994722955145|
|CDLGRAVESTONEDOJI| 0.49532418952618457|
|CDLHAMMER| 0.5021872265966754|
|CDLHANGINGMAN| NA|
|CDLHARAMI| 0.5016807696765967|
|CDLHARAMICROSS| 0.5284249767008388|
|CDLHIGHWAVE| 0.4976385484545944|
|CDLHIKKAKE| 0.4702707485400814|
|CDLHIKKAKEMOD| 0.4547945205479452|
|CDLHOMINGPIGEON| 0.5374554102259215|
|CDLIDENTICAL3CROWS| NA|
|CDLINNECK| NA|
|CDLINVERTEDHAMMER| 0.5385985748218527|
|CDLKICKING| NA|
|CDLKICKINGBYLENGTH| NA|
|CDLLADDERBOTTOM| 0.484375|
|CDLLONGLEGGEDDOJI| 0.503790148219542|
|CDLLONGLINE| 0.47823277128253333|
|CDLMARUBOZU| 0.4736622073578595|
|CDLMATCHINGLOW| 0.4993608862377503|
|CDLMATHOLD| 0.6666666666666666|
|CDLMORNINGDOJISTAR| 0.42769857433808556|
|CDLMORNINGSTAR| 0.4146981627296588|
|CDLONNECK| NA|
|CDLPIERCING| 0.4808743169398907|
|CDLRICKSHAWMAN| 0.5107363131043935|
|CDLRISEFALL3METHODS| 0.5483870967741935|
|CDLSEPARATINGLINES| 0.42602892102335926|
|CDLSHOOTINGSTAR| NA|
|CDLSHORTLINE| 0.4559729567635353|
|CDLSPINNINGTOP| 0.49722797161442933|
|CDLSTALLEDPATTERN| NA|
|CDLSTICKSANDWICH| 0.4005602240896359|
|CDLTAKURI| 0.4860824742268041|
|CDLTASUKIGAP| 0.4820936639118457|
|CDLTHRUSTING| NA|
|CDLTRISTAR| 0.43231441048034935|
|CDLUNIQUE3RIVER| 0.45977011494252873|
|CDLUPSIDEGAP2CROWS| NA|
|CDLXSIDEGAP3METHODS| 0.4615926708949965|

|Bearish Candle | Probability of success (tolerance T+1) |
|---|---|
|CDL2CROWS| 0.47619047619047616|
|CDL3BLACKCROWS| 0.43902439024390244|
|CDL3INSIDE| 0.45104895104895104|
|CDL3LINESTRIKE| 0.5031645569620253|
|CDL3OUTSIDE| 0.4258383490971625|
|CDL3STARSINSOUTH| NA|
|CDL3WHITESOLDIERS| NA|
|CDLABANDONEDBABY| 0.42857142857142855|
|CDLADVANCEBLOCK| 0.4509545753785385|
|CDLBELTHOLD| 0.48220999291668937|
|CDLBREAKAWAY| 1.0|
|CDLCLOSINGMARUBOZU| 0.49671561051004637|
|CDLCONCEALBABYSWALL| NA|
|CDLCOUNTERATTACK| 0.38461538461538464|
|CDLDARKCLOUDCOVER| 0.5505882352941176|
|CDLDOJI| NA|
|CDLDOJISTAR| 0.48736600306278716|
|CDLDRAGONFLYDOJI| NA|
|CDLENGULFING| 0.5245827695083446|
|CDLEVENINGDOJISTAR| 0.5009596928982726|
|CDLEVENINGSTAR| 0.5244807121661721|
|CDLGAPSIDESIDEWHITE| 0.4339622641509434|
|CDLGRAVESTONEDOJI| NA|
|CDLHAMMER| NA|
|CDLHANGINGMAN| 0.49727520435967304|
|CDLHARAMI| 0.46302361661782454|
|CDLHARAMICROSS| 0.4445037353255069|
|CDLHIGHWAVE| 0.4814366912565002|
|CDLHIKKAKE| 0.493694685192828|
|CDLHIKKAKEMOD| 0.5464285714285714|
|CDLHOMINGPIGEON| NA|
|CDLIDENTICAL3CROWS| 0.5727170236753101|
|CDLINNECK| 0.48823529411764705|
|CDLINVERTEDHAMMER| NA|
|CDLKICKING| NA|
|CDLKICKINGBYLENGTH| NA|
|CDLLADDERBOTTOM| NA|
|CDLLONGLEGGEDDOJI| NA|
|CDLLONGLINE| 0.480019195532676|
|CDLMARUBOZU| 0.4906211562115621|
|CDLMATCHINGLOW| NA|
|CDLMATHOLD| NA|
|CDLMORNINGDOJISTAR| NA|
|CDLMORNINGSTAR| NA|
|CDLONNECK| 0.421875|
|CDLPIERCING| NA|
|CDLRICKSHAWMAN| NA|
|CDLRISEFALL3METHODS| 0.5789473684210527|
|CDLSEPARATINGLINES| 0.575|
|CDLSHOOTINGSTAR| 0.4926727145847872|
|CDLSHORTLINE| 0.4716148903000114|
|CDLSPINNINGTOP| 0.481051323033209|
|CDLSTALLEDPATTERN| 0.4568245125348189|
|CDLSTICKSANDWICH| NA|
|CDLTAKURI| NA|
|CDLTASUKIGAP| 0.5245901639344263|
|CDLTHRUSTING| 0.4918918918918919|
|CDLTRISTAR| 0.5147679324894515|
|CDLUNIQUE3RIVER| NA|
|CDLUPSIDEGAP2CROWS| 0.4482758620689655|
|CDLXSIDEGAP3METHODS| 0.47647409172126265|

In [ ]:
def detect_fractals(df):
    df['fractal'] = 0
    
    for i in range(2,df.shape[0]-2):
        if df['low'][i] < df['low'][i-1]  and df['low'][i] < df['low'][i+1] and df['low'][i+1] < df['low'][i+2] and df['low'][i-1] < df['low'][i-2]:
            #df['fractal'][i] = 1.0
            df.iloc[i, df.columns.get_loc('fractal')] = 1.0
        elif df['high'][i] > df['high'][i-1]  and df['high'][i] > df['high'][i+1] and df['high'][i+1] > df['high'][i+2] and df['high'][i-1] > df['high'][i-2]:
            #df['fractal'][i] = -1.0
            df.iat[i, df.columns.get_loc('fractal')] = -1.0
    return df

In [ ]:
#Compute efficacy of fractals patterns as well

signal_frequency = {'bullish': {}, 'bearish':{}}
success_frequency = {'bullish': {}, 'bearish':{}}

candle_names = ['fractal']
for candle in candle_names:
    signal_frequency['bullish'][candle] = 0
    success_frequency['bullish'][candle] = 0
    signal_frequency['bearish'][candle] = 0
    success_frequency['bearish'][candle] = 0
    
    
for stock in Stock.objects.all():
    #listing = get_stock_listing(stock, duration=-1, last_date = datetime.date.today(), resample=False)
    listing = get_stock_listing(stock, duration=-1, last_date = datetime.date.today(), resample=False, monthly=False) #For weekly/monthly charts
    if len(listing)==0:
        continue
    for candle in candle_names:
        df = detect_fractals(listing)
        evaluate_signal(listing, candle, signal_frequency, success_frequency, tolerance=3)
        
print('Bullish')
for key in success_frequency['bullish']:
    if signal_frequency['bullish'][key] !=0:
        print(f"{key}: {success_frequency['bullish'][key]/signal_frequency['bullish'][key]}")
    else:
        print(f'{key}: NA')
print('Bearish')
for key in success_frequency['bearish']:
    if signal_frequency['bearish'][key] !=0:
        print(f"{key}: {success_frequency['bearish'][key]/signal_frequency['bearish'][key]}")
    else:
        print(f'{key}: NA')

In [ ]:
print(signal_frequency['bullish']['fractal'])
print(success_frequency['bullish']['fractal'])

print(signal_frequency['bearish']['fractal'])
print(success_frequency['bearish']['fractal'])